In [2]:
import pandas as pd
from nltk.corpus import stopwords
import json
import numpy as np
import nltk


In [90]:
annotations_data = pd.read_csv('/Users/vcroopana/Downloads/summer2020/superbowl/ip/SB_ad_annotations.csv', index_col=0) 
annotations_data['Keywords'] = annotations_data['Brand Name']\
                                .str.cat(annotations_data['Ad Name'], sep=" ")\
                                .str.cat(annotations_data['Key Terms  Round 1'], sep=" ")
df = annotations_data.drop_duplicates()
print(df.shape)
annotations_data.head(2)


(75, 9)


,Brand Name,Ad Name,Product,Key Terms Round 1,Excitatory Potential,Emotional vs. Rational,Semantic Affinity,Valence,Keywords
Ad Number,,,,,,,,,
1,Trailer,Fast & Furious 9 Trailer,Movie Trailer,"fast and the furious, fast & the furious, fast...",1,1,2,1,Trailer Fast & Furious 9 Trailer fast and the...
2,Quibi,Quibi Bank Heist,Video Platform,"quibi, bank heist, robbery, less than ten minu...",2,1,2,2,"Quibi Quibi Bank Heist quibi, bank heist, rob..."


In [42]:
stop = stopwords.words('english')
print(len(stop))
#stop.extend(["best", "award", "love", "movie", "picture", "words", "actor", "actress", "red", "carpet", "oscar", "oscars"])
print(len(stop))

179
179


In [43]:
#clean tweets
import re
def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac    
    text = textBeforeMention+ " " + textAfterMention  
    return text
    
#TODO issue should've getting converted to shouldve couldn' t -> couldn
def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])    
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', ' ', text) # Remove non-ASCII chars.
    text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K") )

cadillacabc joinrt k


In [5]:
######## GuidedLDA #############

https://tedunderwood.com/2012/04/07/topic-modeling-made-just-simple-enough/
<br /> topic modelling Vs classificaton: https://monkeylearn.com/topic-analysis/#papers
<br />shorttext - https://shorttext.readthedocs.io/en/latest/tutorial_topic.html
<br />naive Bayes - https://monkeylearn.com/topic-analysis/
<br />guidedlda - https://guidedlda.readthedocs.io/en/latest/
<br />guided LDA installation: https://github.com/vi3k6i5/GuidedLDA/issues/26
<br /> guide LDA explanation: https://www.freecodecamp.org/news/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164/

In [6]:
# Normal LDA without seeding
# model = guidedlda.GuidedLDA(n_topics=14, n_iter=100, random_state=7, refresh=20)
# model.fit(X)


In [ ]:
annotations_data['keywords_clean'] = annotations_data['Key Terms  Round 1'].apply(lambda x: cleanTweet(x))
annotations_data['keywords_clean'].apply(lambda x: print(str(x.split())+","))


In [94]:
seed_topic_list = [
    ['fast', 'furious', 'fast', 'furious', 'fast', 'furious', 'ff', 'fast', 'saga', 'vin', 'diesel', 'flying', 'truck', 'stunts', 'michelle', 'rodriguez', 'fast', 'furious'],
['quibi', 'bank', 'heist', 'robbery', 'less', 'ten', 'minutes', 'quick', 'bites', 'big', 'stories', 'chance', 'rapper', 'megan', 'thee', 'stallion', 'chancetherapper', 'meganstallion', 'quickbites'],
['tide', 'laundry', 'laundry', 'detergent', 'schitt', 'emily', 'hampshire', 'charlie', 'day', 'walts'],
['fox', 'show', 'daytona', 'run', 'history', 'great', 'american', 'race'],
['donald', 'trump', 'trump', 'change', 'unemployment', 'stronger', 'safer', 'prosperous', 'trumpforpresident', 'donaldtrump', 'presidentrump', 'obama', 'gop', 'trumpsupporters'],
['walmart', 'pickup', 'spaceship', 'toy', 'story', 'buzz', 'lightyear', 'marvin', 'martian', 'marvin', 'martians', 'arrival', 'glass', 'cleaners', 'aliens', 'men', 'black', 'groot', 'lego', 'star', 'wars', 'bill', 'bill', 'ted', 'flash', 'gordon'],
['marvel', 'black', 'widow', 'scarlett', 'johansson'],
['quicken', 'loans', 'aquaman', 'jason', 'momoa', 'rocket', 'mortgage', 'momoabowl', 'jasonmomoa', 'rocketmorgage'],
['porsche', 'volkswagen', 'chase', 'fast', 'furious', 'taycan', 'turbo', 'car', 'chase', 'taycanturbo'],
['snickers', 'snickersfixtheworld', 'fix', 'world', 'snickershole', 'fixtheworld'],
['hulu', 'tom', 'brady', 'tombrady'],
['fox', 'chosen', 'lego', 'masters', 'competition', 'arnett'],
['mountain', 'dew', 'mtn', 'dew', 'soda', 'zero', 'sugar', 'shining', 'work', 'play', 'makes', 'jack', 'dull', 'boy', 'new', 'mtn', 'dew', 'zero', 'sugar', 'bryan', 'cranston', 'mtndew', 'theshining'],
['squarespace', 'winona', 'winona', 'ryder', 'minnesota'],
['love', 'takes', 'action', 'greek', 'words', 'love', 'agape', 'new', 'york', 'life', 'take', 'action', 'lovetakesaction', 'good', 'life', 'nyl', 'newyorklife'],
['fox', 'super', 'monday', 'office', 'toby', 'toby', 'flenderson'],
['chris', 'evans', 'captain', 'america', 'john', 'krasinki', 'rachel', 'dratch', 'boston', 'red', 'sox', 'david', 'ortiz', 'boston', 'accent', 'smart', 'parking', 'hyundai', 'better', 'way', 'park', 'bettahdriveus', 'chrisevans', 'smahtpark', 'bettahwaytopark'],
['mc', 'hammer', 'cant', 'touch', 'popcorn', 'cheetos', 'cheetos', 'thing', 'cheetle', 'canttouchthis', 'mchammer'],
['female', 'spacewalk', 'lilly', 'singh', 'busy', 'phillips', 'nicole', 'stott', 'taraji', 'henson', 'katie', 'couric', 'olay', 'make', 'space', 'women', 'makespaceforwomen', 'olay'],
['fox', 'halftime', 'show', 'lo', 'jennifer', 'lopez', 'shakira'],
['help', 'farmers', 'organic', 'pack', 'square', 'foot', 'make', 'impact', 'michelob', 'gold', 'michelobgold', 'pack', 'sixforsixpack'],
['molly', 'ringwald', 'avocados', 'mexico', 'avonetwork', 'avocarrier', 'mollyringwald'],
['hard', 'rock', 'international', 'hard', 'rock', 'stadium', 'miami', 'kobe', 'bryant', 'jennifer', 'lopez', 'jlo', 'hardrock', 'hardrockstadium', 'hardrockcasino'],
['rick', 'morty', 'adult', 'swim', 'pringles', 'pickle', 'rick', 'rick', 'morty', 'new', 'flavor', 'summer', 'rick', 'morty', 'adultswim'],
['turbotax', 'people', 'app', 'mobile', 'feature', 'taxes', 'tax', 'people', 'people', 'tax', 'people', 'allpeoplearetaxpeople'],
['tide', 'laundry', 'laundry', 'detergent', 'charliey', 'day', 'bud', 'knight', 'stain', 'laundrydetergent', 'charlieday'],
['john', 'legend', 'chrissy', 'teigen', 'genesis', 'hyundai', 'kobe', 'bryant', 'young', 'luxury', 'gv', 'sexiest', 'man', 'alive', 'johnlegend', 'chrissyteigen'],
['coke', 'iphone', 'coca', 'cola', 'coca', 'cola', 'energy', 'showup', 'dots', 'cococola', 'cocacolaenergy', 'jonah', 'hill', 'martin', 'scorsese'],
['planters', 'nuts', 'mrpeanut', 'matt', 'walsh', 'wesley', 'snipes', 'babynut', 'babypeanut', 'babyfuneral', 'baby', 'nut'],
['james', 'bond', 'time', 'die', 'daniel', 'craig'],
['fox', 'masked', 'singer', 'toads'],
['google', 'assistant', 'virtual', 'assistant', 'googleassistant', 'virtualassistant'],
['sabra', 'humus', 'hummus', 'wives', 'real', 'housewives', 'new', 'jersey', 'real', 'housewives', 'caroline', 'manzo', 'teresa', 'giudice', 'pain', 'mix', 'cracker', 'kim', 'chi', 'drag', 'queens', 'dragqueens', 'dragqueen', 'amanda', 'cerny', 'howimmus'],
['weathertech', 'pets', 'golden', 'retriever', 'dog'],
['verizon', 'phone', 'service', 'phone'],
['general', 'motors', 'hummer', 'hummer', 'ev', 'electric', 'pickup', 'zero', 'emission', 'lebron', 'james', 'basketball', 'lebronjames'],
['pop', 'tarts', 'queer', 'eye', 'jonathan', 'van', 'ness', 'pretzel', 'queereye', 'poptarts'],
['minions', 'rise', 'gru'],
['fox', 'halftime', 'show', 'lo', 'jennifer', 'lopez', 'shakira'],
['fox', 'fox', 'nation', 'breaking', 'news'],
['premier', 'boxing', 'champions', 'wilder', 'vs', 'fury', 'ii'],
['homeland', 'trailer'],
['fox', 'daytona', 'great', 'american', 'race', 'run', 'history'],
['hunters', 'nazi', 'al', 'pacino', 'nazi', 'hunters', 'prime', 'video', 'amazon', 'barbeque', 'barbecue', 'alpacino', 'primevideo', 'hunterstv'],
['missy', 'elliott', 'e', 'pepsi', 'zero', 'sugar', 'paint', 'black', 'pepsi', 'pepsizero', 'nocoke'],
['find', 'goodness', 'heinz', 'ketchup', 'stranger', 'things', 'strangerthings', 'findthegoodness'],
['premier', 'boxing', 'champions', 'wilder', 'vs', 'fury', 'ii', 'wrestling'],
['bud', 'light', 'bud', 'light', 'seltzer', 'post', 'malone', 'anheuser', 'busch', 'inbev', 'hard', 'seltzer', 'postmalone', 'budlight', 'budweiser'],
['office', 'rainn', 'wilson', 'delivery', 'best', 'thing', 'since', 'sliced', 'bread', 'pizza', 'pizza', 'pizza', 'littlecaesars', 'caesars'],
['sam', 'elliott', 'lil', 'nas', 'old', 'town', 'road', 'doritos', 'cowboy', 'cool', 'ranch', 'dancer', 'billy', 'ray', 'cyrus', 'wild', 'west', 'wild', 'wild', 'west', 'make', 'move'],
['josh', 'jacobs', 'running', 'back', 'kia', 'kia', 'seltos', 'raiders', 'give', 'everything', 'joshjacobs', 'kiaseltos'],
['turkish', 'airlines', 'airline', 'flight', 'astronauts', 'rocket'],
['reese', 'take', 'five', 'take', 'candybar', 'reeses'],
['tide', 'stain', 'later', 'charlie', 'day', 'wonder', 'woman'],
['ellen', 'degeneres', 'alexa', 'porta', 'de', 'rossi', 'maid', 'medieval', 'dragon', 'amazon', 'alexa', 'middle', 'ages', 'beforealexa'],
['anheuser', 'busch', 'inbev', 'michelob', 'ultra', 'beer', 'jimmy', 'fallon', 'working', 'gym', 'john', 'cena', 'usain', 'bolt', 'brooks', 'koepka', 'kerri', 'walsh', 'jennings', 'worth', 'enjoy', 'low', 'carbs', 'jimmyfallon', 'usainbolt', 'workingout', 'gymbody'],
['inspire', 'change', 'police', 'shootings', 'black', 'men', 'nfl', 'inspirechange'],
['toyota', 'car', 'suv', 'highlander', 'toyotahighlander', 'cobiesmulders', 'cobie', 'smulders'],
['pop', 'culture', 'annual', 'fee', 'discover', 'credit', 'card', 'chandler', 'friends', 'jack', 'black', 'mike', 'myers', 'austin', 'powers', 'noannualfee', 'discovercredit', 'yestodiscover'],
['disney', 'plus', 'falcon', 'wanda', 'vision', 'wanda', 'loki'],
['pop', 'culture', 'discover', 'credit', 'card', 'mean', 'girls', 'ted', 'noannualfee', 'discovercredit', 'nodiscover'],
['football', 'withdrawal', 'system', 'science', 'doctor', 'withdrawal', 'symptoms'],
['mobile', 'mama', 'tests', 'g', 'anthony', 'anderson'],
['fox', 'tornado', 'lone', 'star'],
['fox', 'masked', 'singer', 'lego', 'masters'],
['nfl', 'building', 'better', 'game', 'amazon', 'web', 'services'],
['kathryn', 'bigelow', 'budweiser', 'american', 'spirit', 'typical', 'american', 'kindness', 'beer', 'stereotypes', 'stereotypical', 'typicalamerican', 'americanspirit'],
['procter', 'gamble', 'come', 'together', 'sofia', 'vergara', 'chili', 'bounty', 'old', 'spice', 'head', 'shoulders', 'olay', 'charming', 'febreze'],
['fox', 'daytona', 'great', 'american', 'race', 'run', 'history'],
['woman', 'coach', 'ers', 'katie', 'sowers', 'surface', 'female', 'coach', 'microsoft', 'first', 'female', 'coach', 'katiesowers', 'erscoach', 'microsoftsurface', 'femalecoach'],
['fox', 'lone', 'star', 'storm', 'tornado'],
['jeep', 'groundhog', 'day', 'jeep', 'gladiator', 'bill', 'murray'],
['chris', 'rock', 'sylvester', 'stallone', 'groups', 'facebook', 'rock', 'music', 'stonehenge', 'rock', 'rocket', 'chrisrock'],
['tide', 'tide', 'pod', 'laundry', 'detergent', 'emily', 'hampshire', 'charlie', 'day', 'finally', 'later'],
['maisie', 'williams', 'game', 'thrones', 'let', 'go', 'frozen', 'audi', 'etron', 'sportback', 'traffic', 'letitgo', 'maisie', 'williams', 'gameofthrones']
]

In [154]:
import guidedlda as guidedlda
import numpy as np

def buildWordVector(train_data, word2id):
    X = np.zeros((len(train_data), len(word2id)), dtype = 'int64')

    for i in range(0, len(train_data)):
        doc_words = train_data[i].split()
        for word in doc_words:  
            if word in word2id:
                X[i][word2id[word]] += 1
    return X

def buildGuidedLDAModel(train_data, seed_topic_list, n_topics, seed_confidence, 
                        n_iter, n_top_words, random_state, refresh):
    
    ad_words = []
    for keywrds in train_data:
        keywordsSet = list(set(keywrds.split()))
        ad_words.extend(keywordsSet)

    ad_words = list(set(ad_words))
    print("Length of ad keywords: "+ str(len(ad_words)))

    word2id = dict((v, idx) for idx, v in enumerate(ad_words)) #dict of word: id

    X = buildWordVector(train_data, word2id)     #Build X - input data

    seed_topics = {}     #Build Seed topics list
    for t_id, st in enumerate(seed_topic_list):
        for word in st:
            if word in word2id:
                 seed_topics[word2id[word]] = t_id

    model = guidedlda.GuidedLDA(n_topics, n_iter=n_iter, random_state= random_state, refresh= refresh)
    model.fit(X, seed_topics=seed_topics, seed_confidence= seed_confidence)

    topic_vector = model.topic_word_ #topic vector [[0.00059559 0..0.00059559]..[0.00263852 ...0.00263852]]
    topic_words = []

    for i, topic_dist in enumerate(topic_vector):
         topic_word = np.array(ad_words)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
         topic_words.append(topic_word.tolist())
         print('Topic {}: {}'.format(i, ' '.join(topic_word)))
    
    return model, topic_words, word2id

In [156]:
#ip required: n_topics, topic_words

def buildTopicVectors(doc_topic, n_topics, topic_words):
    
    columns_label = ['topic {}'.format(i) for i in range(n_topics)]  # number of topics
    topic_vector = pd.DataFrame(doc_topic, columns = columns_label)  #dataframe of doc-topics

    topic_vector['max_prob'] = topic_vector.max(axis=1)
    topic_vector['max_topic'] = topic_vector.idxmax(axis =1)
    topic_vector['max_topic_id'] = topic_vector['max_topic'].apply(lambda x: topic_vector.columns.get_loc(x))
    topic_vector['topic_words'] = topic_vector['max_topic_id'].apply(lambda x: topic_words[x])
    
    return topic_vector

def get_conf_matrix(ad_manual, max_prob, thresh_prob):
    res = ""
    if(ad_manual=='none' and max_prob >= thresh_prob):
        res = 'FP'
    elif(ad_manual=='none' and max_prob < thresh_prob):
        res = 'TN'
    elif(ad_manual!='none' and max_prob >= thresh_prob):
        res = 'TP'
    elif(ad_manual!='none' and max_prob < thresh_prob):
        res = 'FN'
    return res

def computeAccuracy(result):    
    n_tp = result[result['conf_matrix'] == 'TP'].shape[0]
    n_fp = result[result['conf_matrix'] == 'FP'].shape[0]
    n_fn = result[result['conf_matrix'] == 'FN'].shape[0]
    n_tn = result[result['conf_matrix'] == 'TN'].shape[0]
    
    print("n_tp:" + str(n_tp)+ " n_fp:" + str(n_fp)+ " n_fn:" + str(n_fn) + " n_tn:" + str(n_tn))

    precision = n_tp/(n_tp+ n_fp)
    recall = n_tp/(n_tp+ n_fn)
    f_measure = (2*precision*recall)/ (precision+recall)

    return precision, recall, f_measure


def removePOS(taggedText, includePOS):
    op = ' '.join([pair[0] for pair in taggedText if pair[1] in includePOS])
    return op


In [212]:
#training data for the model
import logging

main = logging.getLogger()
main.setLevel(logging.ERROR)

train_data = annotations_data['keywords_clean'].apply(lambda x: cleanTweet(x))
print(len(train_data))
#Filter only adjectives and nouns
train_data_pos = train_data.apply(lambda x: removePOS(nltk.pos_tag(x.split()), ('NN') )).to_list()

seed_topic_pos = []
for seeds in seed_topic_list:
    tagged = nltk.pos_tag(seeds)
    seed_topic_pos.append(removePOS(tagged, ( 'NN')))

model, topic_words, word2id = buildGuidedLDAModel(
#                                                   train_data.to_list(), 
                                                train_data_pos, 
#                                                   seed_topic_list,
                                                seed_topic_pos,
#     [],
                                                  n_topics= 75, 
                                                  seed_confidence = 0.7, 
                        n_iter= 100, n_top_words = 7, random_state=7, refresh=1)


75
Length of ad keywords: 338
Topic 0: yestodiscover fee mike gop mountain jacobs life
Topic 1: way gladiator tom hulu sox murray john
Topic 2: trump gop alpacino trumpforpresident dog jacobs life
Topic 3: jonah energy iphone gop mountain jacobs life
Topic 4: rocket chris michelob stallone music stonehenge gop
Topic 5: swim game emission gop stonehenge life unemployment
Topic 6: bud findthegoodness busch budweiser malone anheuser post
Topic 7: jimmy jimmyfallon walsh gym usainbolt gymbody john
Topic 8: donaldtrump presidentrump unemployment change dog hardrock jacobs
Topic 9: beer busch inbev cena michelob fallon gop
Topic 10: chase taycanturbo car turbo porsche mountain jacobs
Topic 11: day detergent charlie tax pod tide schitt
Topic 12: life love gop mountain hardrock jacobs unemployment
Topic 13: minnesota gop dog hardrock jacobs life unemployment
Topic 14: winona squarespace gru ryder mountain hardrock jacobs
Topic 15: lil town move road dancer trailer homeland
Topic 16: chancether

In [ ]:
man_ann_data = pd.read_csv(r'/Users/vcroopana/Downloads/summer2020/superbowl/mann_ann_sb.csv')    
man_ann_data.head(1)

In [198]:
man_ann_data['text_clean'] = man_ann_data['tweet_text'].apply(lambda x: cleanTweet(x))
test_data = man_ann_data['text_clean'] 
X = buildWordVector(test_data, word2id)

doc_topic = model.transform(X)
# print(doc_topic)
topic_vector = buildTopicVectors(doc_topic, 75, topic_words)
# topic_vector.round(2).head(2)

man_ann_data = man_ann_data.reset_index(drop=True)

result = pd.concat([man_ann_data, topic_vector], axis=1)


In [204]:
result['conf_matrix'] = result.apply(lambda x: get_conf_matrix(x['ad_manual'], x['max_prob'], 
                                                               thresh_prob = 0.54), axis =1)
precision, recall, f_measure = computeAccuracy(result)
print("precision: "+ str(np.round(precision, 3))+" recall: "+ str(np.round(recall,3))+" f_measure: "
      + str(np.round(f_measure,3)))

n_tp:629 n_fp:962 n_fn:224 n_tn:685
precision: 0.395 recall: 0.737 f_measure: 0.515


In [ ]:
result.head(1)

In [188]:
op_name = '/Users/vcroopana/Downloads/summer2020/superbowl/op_mann_lda.csv'
cols_reqd = ['tweet_text', 'text_clean', 'ad_mentioned', 'ad_keywords', 'ad_manual', 'max_prob', 'max_topic_id', 'topic_words', 'conf_matrix']
result[cols_reqd].to_csv(op_name)


In [ ]:
# read random 2500 tweets from given list

In [ ]:

sb_tweets = pd.read_csv(r'/Users/vcroopana/Downloads/summer2020/superbowl/ip/SB2020_tweets_affectivestate.csv')    
print(sb_tweets.shape)
sb_tweets.head(5)


In [65]:
import random
def getRandomInts(nRandInts, rangeStart, rangeEnd):
    randomlist = []
    for i in range(0,nRandInts):
        n = random.randint(rangeStart,rangeEnd)
        randomlist.append(n)
    return randomlist

def retrieveRandList(randListFile):
    randDf = pd.read_csv(randListFile) 
    randList = randDf['random'].values.tolist()
    return randList

randList = getRandomInts(2500, 0, 110735)
randList_filename = "/Users/vcroopana/Downloads/summer2020/superbowl/randList_2_5_K.csv"
pd.DataFrame(randList, columns = ['random']).to_csv(randList_filename) #Save list to file
randTweets = sb_tweets.iloc[randList]
randTweets.to_csv("/Users/vcroopana/Downloads/summer2020/superbowl/mann_ann_sb.csv")

In [53]:
test_data = [cleanTweet(doc) for doc in man_ann_data['tweet_text']]
X = buildWordVector(test_data, word2id)
print("Size of test data:"+ str(X.shape))

doc_topic = model.transform(X)
topic_vector = buildTopicVectors(doc_topic, 75, topic_words)

man_ann_data = man_ann_data.reset_index(drop=True)
result = pd.concat([man_ann_data, topic_vector], axis=1)


(110735, 682)


In [61]:
result['conf_matrix'] = result.apply(lambda x: get_conf_matrix(x['ad_mentioned'], x['max_prob'], 
                                                               thresh_prob = 0.1), axis =1)
precision, recall, f_measure = computeAccuracy(result)
print("precision:"+ str(np.round(precision,3))+" recall:"+ str(np.round(recall,3))
      +" f_measure:"+ str(np.round(f_measure,3)))


n_tp:110421 n_fp:0 n_fn:314
precision:1.0 recall:0.997 f_measure:0.999


In [ ]:

#normal lda

In [206]:
import pyLDAvis.gensim
from gensim import models

def buildDict(train_data):
    ad_words = []
    for keywrds in train_data:
        keywordsSet = list(set(keywrds.split()))
        ad_words.extend(keywordsSet)

    ad_words = list(set(ad_words))
    print("Length of ad keywords: "+ str(len(ad_words)))

    word2id = dict((v, idx) for idx, v in enumerate(ad_words)) #dict of word: id
    return word2id


def displayModel(model, corpus, dictionary):
    
    lda_model_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True, mds='mmds')
    pyLDAvis.show(lda_model_display)

In [208]:

def sort_tuple(tupList):  
    tupList.sort(reverse = True, key = lambda x: x[1])  
    return tupList  

def printTopics(model, nTopics, nWords):
#     topics = list(set(model.show_topics(nTopics, 5)))
#     topics.sort(key = lambda x: x[1])
    for i in range(0, nTopics):
        topic = model.show_topic(i, 100)  
        sort_tuple(topic)
        print(str(i)+":"+ str(topic[0:nWords]))
        
def get_lda_model(nTopics, ad_corpus, ad_dictionary, annotations_data):
    
#     annotations_data['keywords_clean'] = annotations_data['Keywords'].apply(lambda x: cleanTweet(x))
    n_words = len(ad_dictionary) #379, 40k - for 2lakh tweets

    model = models.ldamodel.LdaModel(ad_corpus, num_topics= nTopics, 
                                     id2word = ad_dictionary, 
    #                                  chunksize= 74, 
                                     alpha = 'auto', 
#                                      passes=50,
#                                      eta = topic_word_priors,
#                                      per_word_topics = True
#                                      minimum_phi_value = 0.001
    #                                  minimum_probability = 0.000001
                                    ) 
    topics = model.print_topics(nTopics, num_words= 100)
    print(len(ad_dictionary))
    print(len(ad_corpus))
    print(len(topics))
    printTopics(model, nTopics, nWords = 5)
    return model

In [209]:
# displayModel(model, , word_dict)

from gensim import corpora

test_data = [cleanTweet(doc) for doc in man_ann_data['tweet_text']]

tweet_words = [tweet.split() for tweet in test_data]
corpora_dict = corpora.Dictionary(tweet_words)
tweet_corpus = [corpora_dict.doc2bow(text) for text in tweet_words]

lda_model = get_lda_model(75, tweet_corpus, corpora_dict, annotations_data)


5309
2500
75
0:[('shakira', 0.04269479), ('grow', 0.028393073), ('fact', 0.027062088), ('los', 0.027044864), ('pepsihalftime', 0.024754902)]
1:[('ers', 0.031258497), ('tarts', 0.03098391), ('pop', 0.030983828), ('pretzel', 0.030983007), ('cry', 0.029526664)]
2:[('announcement', 0.063551664), ('tombrady', 0.061657645), ('ad', 0.061657395), ('brady', 0.022319697), ('tom', 0.022318989)]
3:[('nba', 0.032000337), ('petition', 0.031205203), ('kobe', 0.025993345), ('going', 0.024465458), ('bryant', 0.024464268)]
4:[('mathieu', 0.049486637), ('th', 0.043711223), ('superbowl', 0.036069967), ('football', 0.029940888), ('star', 0.024757495)]
5:[('nfl', 0.06172497), ('app', 0.049421523), ('fox', 0.04006005), ('sbliv', 0.02682572), ('sports', 0.025650632)]
6:[('take', 0.04876036), ('miami', 0.04476696), ('left', 0.03801245), ('halftime', 0.03782972), ('show', 0.036236554)]
7:[('two', 0.057215206), ('next', 0.03667879), ('mathieu', 0.03503913), ('era', 0.035037976), ('niners', 0.02101851)]
8:[('road

In [210]:
displayModel(lda_model, tweet_corpus, corpora_dict)




Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8891/    [Ctrl-C to exit]


127.0.0.1 - - [23/Jun/2020 19:30:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 19:30:17] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 19:30:17] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 19:30:17] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2020 19:30:17] code 404, message Not Found
127.0.0.1 - - [23/Jun/2020 19:30:17] "GET /favicon.ico HTTP/1.1" 404 -



stopping Server...
